In [ ]:
using Statistics
using CSV
using DataFrames
using LinearAlgebra
using Dates


In [ ]:
filepath = "D:/4.5.csv"
df = CSV.read(filepath, DataFrame, dateformat="dd-mm-yyyy", limit = 10)

In [ ]:
magnitudes = df.mag
println("Min magnitude = [$(minimum(magnitudes))]")
println("Max magnitude = [$(maximum(magnitudes))]")
println("Avg magnitude = [$(sum(magnitudes) / length(magnitudes))]")

In [55]:
for x in df.time
    println("$(x[1:19])")
end
#the loop ends after 10 iterations for some reasons

2023-01-27T20:25:27
2023-01-27T18:18:00
2023-01-27T16:37:39
2023-01-27T16:33:28
2023-01-27T11:06:54
2023-01-27T10:51:18
2023-01-27T05:29:21
2023-01-27T04:34:38
2023-01-27T03:21:35
2023-01-27T00:10:43


"2023-01-27T18:18:00"

In [66]:

function avgMagnitudeNDays(n)  #return error for n>10, similar to the above code chunk behavior
    sumOfMag= 0
    for x in 1:n
        sumOfMag += df.mag[x]
    end
    return sumOfMag/n
end

#for x in [10, 20, 30, 40 ,50] 
    res = avgMagnitudeNDays(10)
    println(res)
#end


4.69


TU PHAM

MATT LE